
# Performance Pandas

![](http://d.pr/i/cLFsU9+)

Python is slower than compiled languages for a variety of reasons:

### Python is Dynamically Typed rather than Statically Typed.

What this means is that at the time the program executes, the interpreter doesn't know the type of the variables that are defined. For example, the difference between a C variable and a Python variable is summarized by this diagram:

![](images/cint_vs_pyint.png)

For a variable in C, the compiler knows the type by its very definition. For a variable in Python, all you know at the time the program executes is that it's some sort of Python object.

So if you write the following in C:

```C
int a = 1;
int b = 2;
int c = a + b;
```

the C compiler knows from the start that a and b are integers: they simply can't be anything else! With this knowledge, it can call the routine which adds two integers, returning another integer which is just a simple value in memory. As a rough schematic, the sequence of events looks like this:

**C Addition**

1. Assign <int> 1 to a
2. Assign <int> 2 to b
3. call binary_add<int, int>(a, b)
4. Assign the result to c

The equivalent code in Python looks like this:

```python
a = 1
b = 2
c = a + b
```

here the interpreter knows only that 1 and 2 are objects, but not what type of object they are. So the The interpreter must inspect PyObject_HEAD for each variable to find the type information, and then call the appropriate summation routine for the two types. Finally it must create and initialize a new Python object to hold the return value. The sequence of events looks roughly like this:

**Python Addition**

1. Assign 1 to a
    - Set a->PyObject_HEAD->typecode to integer
    - Set a->val = 1
2. Assign 2 to b
    - Set b->PyObject_HEAD->typecode to integer
    - Set b->val = 2
3. call binary_add(a, b)
    - find typecode in a->PyObject_HEAD
    - a is an integer; value is a->val
    - find typecode in b->PyObject_HEAD
    - b is an integer; value is b->val
    - call binary_add<int, int>(a->val, b->val)
    - result of this is result, and is an integer.
4. Create a Python object c
    - set c->PyObject_HEAD->typecode to integer
    - set c->val to result

The dynamic typing means that there are a lot more steps involved with any operation. This is a primary reason that Python is slow compared to C for operations on numerical data.

### Python is interpreted rather than compiled.

We saw above one difference between interpreted and compiled code. A smart compiler can look ahead and optimize for repeated or unneeded operations, which can result in speed-ups. Compiler optimization is its own beast, and I'm personally not qualified to say much about it, so I'll stop there. 

### Python's object model can lead to inefficient memory access

We saw above the extra type info layer when moving from a C integer to a Python integer. Now imagine you have many such integers and want to do some sort of batch operation on them. In Python you might use the standard List object, while in C you would likely use some sort of buffer-based array.

A NumPy array in its simplest form is a Python object build around a C array. That is, it has a pointer to a contiguous data buffer of values. A Python list, on the other hand, has a pointer to a contiguous buffer of pointers, each of which points to a Python object which in turn has references to its data (in this case, integers). This is a schematic of what the two might look like:

![](images/array_vs_list.png)

You can see that if you're doing some operation which steps through data in sequence, the numpy layout will be much more efficient than the Python layout, both in the cost of storage and the cost of access.

## Speeding up statistical computations in Python

In the age of "big data" and sophisitcated Bayesian and statistical learning algorithms, many are interested in optimizing the performance of the high-level languages that we use to analyse data.

[NumPy](http://numpy.scipy.org/) gets us part of the way there on Python:

* Storage of multidimensional data
* Efficient data access
* Efficient in-memory storage
* Fast methods and functions for data manipulation

Ffor many applications, this is sufficient to drastically improve performance. However, there is plenty of scope for improving Python's performance in situations where speed matters.

Pure Python and Python with NumPy are not particularly fast. Below are some recent performance benchmarks comparing several computing languages (taken directly from the [Julia website](http://julialang.org)):


<div class="figure">
<table class="benchmarks">
<colgroup>
<col class="name" />
<col class="relative" span="11" />
</colgroup>
<thead>
<tr><td></td><th class="system">Fortran</th><th class="system">Julia</th><th class="system">Python</th><th class="system">R</th><th class="system">Matlab</th><th class="system">Octave</th><th class="system">Mathematica</th><th class="system">JavaScript</th><th class="system">Go</th><th class="system">LuaJIT</th><th class="system">Java</th></tr>
<tr><td></td><td class="version">gcc 5.1.1
</td><td class="version">0.4.0
</td><td class="version">3.4.3
</td><td class="version">3.2.2
</td><td class="version">R2015b
</td><td class="version">4.0.0
</td><td class="version">10.2.0
</td><td class="version">V8 3.28.71.19
</td><td class="version">go1.5
</td><td class="version">gsl-shell 2.3.1
</td><td class="version">1.8.0_45
</td></tr>
</thead>
<tbody>
<tr><th>fib</th><td class="data">0.70</td><td class="data">2.11</td><td class="data">77.76</td><td class="data">533.52</td><td class="data">26.89</td><td class="data">9324.35</td><td class="data">118.53</td><td class="data">3.36</td><td class="data">1.86</td><td class="data">1.71</td><td class="data">1.21</td></tr>
<tr><th>parse_int</th><td class="data">5.05</td><td class="data">1.45</td><td class="data">17.02</td><td class="data">45.73</td><td class="data">802.52</td><td class="data">9581.44</td><td class="data">15.02</td><td class="data">6.06</td><td class="data">1.20</td><td class="data">5.77</td><td class="data">3.35</td></tr>
<tr><th>quicksort</th><td class="data">1.31</td><td class="data">1.15</td><td class="data">32.89</td><td class="data">264.54</td><td class="data">4.92</td><td class="data">1866.01</td><td class="data">43.23</td><td class="data">2.70</td><td class="data">1.29</td><td class="data">2.03</td><td class="data">2.60</td></tr>
<tr><th>mandel</th><td class="data">0.81</td><td class="data">0.79</td><td class="data">15.32</td><td class="data">53.16</td><td class="data">7.58</td><td class="data">451.81</td><td class="data">5.13</td><td class="data">0.66</td><td class="data">1.11</td><td class="data">0.67</td><td class="data">1.35</td></tr>
<tr><th>pi_sum</th><td class="data">1.00</td><td class="data">1.00</td><td class="data">21.99</td><td class="data">9.56</td><td class="data">1.00</td><td class="data">299.31</td><td class="data">1.69</td><td class="data">1.01</td><td class="data">1.00</td><td class="data">1.00</td><td class="data">1.00</td></tr>
<tr><th>rand_mat_stat</th><td class="data">1.45</td><td class="data">1.66</td><td class="data">17.93</td><td class="data">14.56</td><td class="data">14.52</td><td class="data">30.93</td><td class="data">5.95</td><td class="data">2.30</td><td class="data">2.96</td><td class="data">3.27</td><td class="data">3.92</td></tr>
<tr><th>rand_mat_mul</th><td class="data">3.48</td><td class="data">1.02</td><td class="data">1.14</td><td class="data">1.57</td><td class="data">1.12</td><td class="data">1.12</td><td class="data">1.30</td><td class="data">15.07</td><td class="data">1.42</td><td class="data">1.16</td><td class="data">2.36</td></tr>
</tbody>
</table>

<p class="caption"><b>Figure:</b>
benchmark times relative to C (smaller is better, C performance = 1.0).
</p>
</div>

So, while fast relative to some scientific compution choices (*e.g.* R, Matlab), Python sometimes needs to be tweaked in order to make it a competitive choice for implementing modern statistical methods. We will cover two approachable ways of improving the performance of Python.


## Profiling

Before you barrel ahead and prematurely optimize your Python code, it is important to understand **why** and **where** your code is slow. This is achieved by systematically accounting for the resources that your code is using, such as memory, CPU time or data transfer. This process is broadly referred to as ***Profiling***, and it allows you to identify where the performance bottlenecks in your code lie.

Here, we will concentrate on optimizing performance for **CPU-bound** problems.

There are a number of tools to help you profile your code.

### `time`

For those of you on UNIX platforms, the built-in utility `time` can be used to assess how long your code takes to run.

In [ ]:
!time python ../examples/abc.py

The output from `time` can be interpreted as:

* `real`: elapsed (wall) time
* `user`: time spent in your code
* `sys`: time spent in system (kernel) functions

The last 2 quantities account for the cycles used by your program. The remaining `real` time is often due to waiting for information either from disk or a network connection (I/O).

Python also has a `time` module (and function) that is more rudimentary; it simply returns the time, in seconds from the Epoch (1/1/1970).

In [ ]:
import time
time.time()

We can use this for profiling by differencing the times before and after running some code of interest:

In [ ]:
import numpy as np
start_time = time.time()
np.product(range(1, 100000))
end_time = time.time()

end_time - start_time

Note, however that it does not provide a breakdown of where the code spends its time.

### IPython magic: `%timeit`, `%run` and `%prun`

IPython has three built-in "magic" functions that are useful for profiling your code. 

The `%timeit` magic executes a Python statement or expressions in a loop to see how long we expect it to take for any given call. Additionally, it repeats the loop a certain number of times, and returns the best result.

As an example, consider a Python implementation of the **trapezoidal rule**, a method from numerical analysis for approximating a definite integral. Specifically, it allows us to approximate:

$$\int_a^b f(x) dx$$

using the approximation:

$$\int_a^b f(x) dx \approx (b-a) \frac{f(b) + f(a)}{2}$$

Rather than use a single interval for this estimate, we break the interval down into $n$ subintervals, to obtain a more accurate approximation.

In [ ]:
def f(x):
    return 2*x*x + 3*x + 1
      
def trapez(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h

In [ ]:
trapez(1, 5, 10000)

To confirm that this works, we can compare this to the symbolic solution, using Sympy:

In [ ]:
import sympy as sym

xs = sym.symbols('xs')

fx = 2*xs*xs + 3*xs + 1

ifx = sym.integrate(fx, (xs, 1, 5))
ifx.evalf()

In [ ]:
%timeit trapez(1, 5, 10000)

`%timeit` tries to pick suitable values for the number of loops and repeats; these values can be overriden by specifying `-n` and `-r` values, respectively.

Profiling results can be saved to a variable by calling the %timeit magic with the `-o` flag:

    %timeit -o <expression>

This returns a `TimeitResult` object, which includes information about the %timeit run as attributes.

In [ ]:
trapez_prof = %timeit -o trapez(1, 5, 10000)

In [ ]:
trapez_prof.best

The `%run` command with a `-p` option allows you to run complete programs under the control of the Python profiler. It writes the output to the help pane, which opens at the bottom of the page.

In [ ]:
# This code redirects pager output to a regular cell
from IPython.core import page
page.page = print

In [ ]:
%run -p ../examples/abc.py

The profiling information includes the following information:

* `ncalls`: number of calls to function
* `tottime`: total time spent in the given function (excluding time in calls to sub-functions)
* `percall`: time per call
* `cumtime`: cumulative time spent in this and all subfunctions 

We can see that most of the time in this example is spent inside of core NumPy functions and methods.

The `%prun` command does a similar job for single Python expressions (like function calls).

In [ ]:
%prun trapez(2, 6, 100000)

For even more fine-grained profiling information, we can use a line profiler to see how long it takes each line of a function to run.

In [ ]:
!pprofile ../examples/bisection.py

This output makes it clear that the biggest cost is in the repeated calling of the function $f$ for which the root is being found. If we could improve the speed of this function, it would be the easiest single way of improving the performance of the code.


## Speeding up Pandas by Being Idiomatic

When you have decided that your code is unacceptably slow, and have gone through the process of profiling to see if and where your program is experiencing a bottleneck, it can be easy to jump ahead and try speeding it up using external tools. There are several packages that will certainly improve Python's performance (and we will introduce some of them later), but the first place to look for better performance is in **refactoring** your implementation of whichever algorithm you happen to be using. 

Effective pandas programming (and Python, in general) involves applying particular **idioms** effectively; these are idiosyncratic expressions that may only exist in Python or pandas, but when used appropriately they can make your code more readable, faster, or both. You have seen some of these already -- for example, the **list comprehension** as a means for succinctly implementing a `for` loop.

### Comprehensions

In [ ]:
def do_math(x):
    return 3 + x**3

In [ ]:
%%timeit
squares = []
for i in range(1000):
    squares.append(do_math(i))

In [ ]:
%timeit squares = [do_math(i) for i in range(1000)]

Here, not only is the list comprehension easier to write and read, it is also slightly faster.

### String concatenation

Just as you should avoid growing lists or arrays by concatenation or appending, iterating over strings and concatenating them manually is very inefficient. For example, let's say we want to concatente a list of strings into a single string:

In [ ]:
words = ["Six",
"days",
"in",
"to",
"what",
"should",
"be",
"a",
"greatest",
"two",
"months",
"of",
"my",
"life",
"and",
"it’s",
"turned",
"in",
"to",
"a",
"nightmare"]

One might be tempted to code the following:

In [ ]:
%%timeit
sentence = ""
for word in words:
    sentence += word

However, this is inefficient; since strings is immutable in Python, every `+` operation involves creating a new string and copying the old content. Instead, we can use the string method `join`, which is not only faster, but more flexible. Here, we would like to separate the words by spaces, which is easily done:

In [ ]:
' '.join(words)

In [ ]:
%timeit ' '.join(words)

### Concatenating DataFrames

An often-seen pattern in pandas is the combining of several imported datasets into a single DataFrame. Larger datasets are frequently stored in chunks on disk (*e.g.* multiple years of meteorological data).

One might instinctually want to instantiate an empty DataFrame of the appropriate dimension (in terms of columns), and iteratively add data to it. For example, consider the ebola data that we explored in a previous section. The data from Liberia consists of a directory of CSV files with identical structure.

We can use the IPython "bang" syntax to retrieve the list of files from this directory and assign them to a variable as a list.

In [ ]:
DATA_DIR = '../data/ebola/liberia_data/'
data_files = !ls $DATA_DIR

Here are the column names for each file:

In [ ]:
columns = ['Date','Variable','National','Bomi County','Bong County','Grand Kru',
           'Lofa County','Margibi County','Maryland County','Montserrado County',
           'Nimba County','River Gee County','RiverCess County','Sinoe County']

Under this strategy, we create an empty DataFrame and loop over the list of files, appending the contents of the file to the DataFrame. You might already be able to guess that this is not an efficient approach.

In [ ]:
%%timeit
liberia_data = pd.DataFrame(columns=columns)
for f in data_files:
    chunk = pd.read_csv(DATA_DIR+f)
    liberia_data = liberia_data.append(chunk)

In [ ]:
liberia_data.shape

In [ ]:
%%timeit
liberia_data = pd.concat([pd.read_csv(DATA_DIR+f) for f in data_files])

### Iteration and vectorization

In [ ]:
vessels = pd.read_csv("../data/AIS/vessel_information.csv", index_col='mmsi')
segments = pd.read_csv("../data/AIS/transit_segments.csv")
segments_merged = pd.merge(vessels, segments, left_index=True, right_on='mmsi')

In [ ]:
def top(df, column, n=5):
    return df.sort_values(by=column, ascending=False)[:n]

In [ ]:
segments_by_vessel = segments_merged.groupby('mmsi')

In [ ]:
%timeit -n 3 segments_by_vessel.apply(top, column='seg_length', n=3)[['names', 'seg_length']].head()

In [ ]:
%timeit -n 3 segments_by_vessel.seg_length.nlargest(3).head()

### Categorical variables

General advice for gaining speed and efficiency with pandas is to use appropriate data types within columns of a DataFrame or in a Series. When importing data, columns can end up with an `object` data type, which is very general, but also quite inefficient.

In [ ]:
vessels.dtypes

`object` data are manipulated using pure Python code, whereas various numneric types run using faster C code. With character data you are generally stuck with an `object` data type, though there is one exeption: **categorical** data.

Categorical data are strings with relatively few distict values relative to the number of elements in the data (also known as having loe cardinality). In pandas, you may want to represent such data using the `categorical` data type.

For example, consider the `flag` column in the vessel dataset:

In [ ]:
vessels.flag.unique().shape

In [ ]:
vessels.shape

In [ ]:
vessels['flag_cat'] = vessels.flag.astype('category')
vessels.flag_cat.head()

The categories are represented internally by unique integers, which is far more compact to store in memory.

In [ ]:
vessels.flag_cat.memory_usage(index=False)

In [ ]:
vessels.flag.memory_usage(index=False)

Not only are `categorical` variables more memory efficient than leaving them as `object` types, but they can appreciably speed up computations that use them as well.

In [ ]:
segments_merged['flag_cat'] = segments_merged.flag.astype('category')

In [ ]:
%timeit segments_merged.groupby('flag').seg_length.nlargest(10).sum()

In [ ]:
%timeit segments_merged.groupby('flag_cat').seg_length.nlargest(10).sum()

That is an appreciable speedup obtained simply by using a more appropriate data type.

## Fast array expression evaluation with `eval`


Since the performance of processors has outpaced that of memory in the past several decades, the CPU spends a lot of time waiting for memory to give it computations; this is the ***processor-memory performance gap***.

![performance gap](http://www.techdesignforums.com/practice/files/2013/02/tdf-snps-ARMcc-feb13-fig1lg.jpg)
(graph courtesy http://www.techdesignforums.com)

CPU caches are often used to make up for this difference. CPU caches are more effective when the data are optimally located in memory to take advantage of cache performance. `numexpr` does this by moving contiguous blocks of data from memory to the CPU cache, reusing them as much as possible within the cache to more quickly give the CPU access to the data.

The [`numexpr`](http://code.google.com/p/numexpr/) package allows array expressions to be evaluated far faster that what can be achieved in Python using Numpy arrays. `numexpr` parses a string expression and optimizes and compiles the code on the fly, using a virtual machine that includes a [Just-in-time (JIT) compiler](http://en.wikipedia.org/wiki/Just-in-time_compilation). 

In addition, `numexpr` offers direct support for parallel multi-threaded computations, since Python's global interpreter lock is bypassed.

> Python's global interpreter lock (GIL) ensures that only one thread runs in the interpreter at once. This simplifies many of the low-level activities, such as memory management, and allows for co-operative multi-tasking. But, since the currently-running thread holds onto the interpreter, it makes multi-core parallelization difficult.

Part the reason Python can be slow for array calculations is that it creates temporary arrays to store intermediate results from array element calculations, which wastes memory and cache. `numexpr` handles such calculations in manageable chunks, which accellerates computation.

The speedup over NumPy by using `numexpr` can be as high as 20x, but is typically in the range of 2-4x.

### pandas  `eval()`

The `eval` function in pandas implements `numexpr` for as an engine for expression evaluation with `Series` and `DataFrame` objects.

`eval` provides better efficiency for evaluation of large datasets, whereby large expressions are evaluated simultaneously by the `numexpr` engine.

The operations supported include:

- Arithmetic operations except for the left shift (<<) and right shift (>>) operators
    - `df + 2 * pi / s ** 4 % 42 - the_golden_ratio`
- Comparison operations, including chained comparisons
    - `2 < df < df2`
- Boolean operations
    - `df < df2 and df3 < df4 or not df_bool`
- `list` and `tuple` literals
    - `[1, 2] or (1, 2)`
- Attribute access
    - `df.a`
- Subscript expressions
    - `df[0]`
- Math functions: `sin, cos, exp, log, expm1, log1p, sqrt, sinh, cosh, tanh, arcsin, arccos, arctan, arccosh, arcsinh, arctanh, abs` and `arctan2`

Most complex Python syntax is **not** supported, including flow control statements, funciton calls (except math), generator expressions, dictionaries and sets, and lambda functions.

In [ ]:
NROWS, NCOLS = 10000, 1000

df1, df2, df3 = [pd.DataFrame(np.random.randn(NROWS, NCOLS)) for _ in range(3)]

In [ ]:
%timeit df1 + df2 + df3

In [ ]:
%timeit pd.eval('df1 + df2 + df3')

You can use a Python backend to `eval` rather than `numexp`, but it is not generally useful.

In [ ]:
%timeit pd.eval('df1 + df2 + df3', engine='python')

Let's do boolean operations now

In [ ]:
%timeit (df1 > df2) & (df2 > df3)

In [ ]:
%timeit pd.eval('(df1 > df2) & (df2 > df3)')

Valid expressions can also be evaluated using the `DataFrame.eval` method. This allows you to avoid prefixing dataframe names to the columns you want to operate on.

In [ ]:
df = pd.DataFrame(np.random.poisson(lam=10, size=(1000000, 2)), columns=['x', 'y'])

In [ ]:
df.eval('x + y')

In [ ]:
df = pd.DataFrame(np.random.normal(10, scale=5, size=(1000000, 2)), columns=['x', 'y'])

df.eval('1.5*x + 0.25*x**2 - 3.4*y + 0.75*y**2 - 10').head()

You can also use the `eval` method to perform assignment of columns within an expression, provided that the assignment target is a valid Python identifier.

This is one of the rare cases where `inplace=True` is not a bad idea (in fact, its the default).

In [ ]:
df.eval('z = x<10', inplace=True)

In [ ]:
df.head()

Multiple assignment can be achieved by using multi-line strings.

In [ ]:
df.eval('''z = x<10
w = (x**2 + y**2)**0.5''', inplace=False).head()

Local variables can be accessed using the `@` identifier.

In [ ]:
const = 0.001

In [ ]:
df.eval('x * @const').head()

Note that this does not work in the `eval` function.

In [ ]:
pd.eval('df.x + @const')

The larger the DataFrames and/or expression, the bigger gain in performance you will see.

### Exercise

Use both `eval` and Python to operate on data frames of different size, and report the performance of each.

- 0 to 10 million rows in increments of 1 million
- 0 to 50,000 rows in increments of 1000

Save the timings to a DataFrame and plot the relative performance in each case.

In [ ]:
# Write your answer here

## Cython

Python developers typically solve performance constraints by building Python extensions by wrapping code written in other languages (for example, SciPy contains more lines of C/C++/Fortran than Python). However, programming with the Python/C API is not straightforward for most users.

Cython is a language that allows Python programmers to write fast code without having to write C/C++/Fortran directly. It looks much like Python code, but with type declarations. Cython code is translated it to C (or C++ or others), which is then compiled to create a Python extension that we can import and use. 

Using Cython, we can achieve speedups of several orders of magnitude, often *faster than hand-coded C code*. In addtion, Cython is compatible with core scientific programming tools like NumPy and IPython.

Cython has built-in support for multicore processing.

Cython is used to varying degrees by other packages in the Python scientific stack, such as sympy, scikit-learn, SciPy and pandas.

### Example: Numerical integration

Recall from above the function `trapez` for performing numerical integration using the trapezoidal rule. 

```python
def f(x):
    return 2*x*x + 3*x + 1
      
def trapez(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h
```

Let's `apply` this function to a DataFrame of values:

In [ ]:
df = pd.DataFrame({'a': np.random.randn(1000),
                    'b': np.random.randn(1000),
                    'N': np.random.randint(100, 1000, (1000)),
                    'x': 'x'})

In [ ]:
%timeit df.apply(lambda x: trapez(x.a, x.b, x.N), axis=1)

Let's profile this to see where the code is slow.

In [ ]:
%prun -l 4 df.apply(lambda x: trapez(x.a, x.b, x.N), axis=1)

The majority of the time is spent inside either of our two functions, so it is worthwhile to convert them to Cython.

Perhaps the easiest way to use Cython, is via the IPython cython magic, which allows us to run Cython interactively:

In [ ]:
%load_ext Cython

Let's simply apply this magic to the functions as written, without changing anything.

In [ ]:
%%cython

def f(x):
    return 2*x*x + 3*x + 1

def trapez2(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h

The Cython magic is doing a lot of work for you: it compiles the code into an extension module, and loads it into the notebook. This allows us to ignore all of the compilation details of building Cython extensions. 

If we run `trapez2`, we can see a reasonable speedup simply by compiling it, unchanged, using Cython.

In [ ]:
%timeit df.apply(lambda x: trapez2(x.a, x.b, x.N), axis=1)

Under the hood, several things are happening in order to deliver this improved performance. The Cython source code is translated into C source code by `cython`. Then, this C source is compiled, using the appropriate compiler, flags and associated library files (if any), into a Python extension. This extension is then loaded by IPython into the current session.

![cython flow](images/cython.png)

C extensions can also be compiled manually, using a setup file. Here is an example for an extension called `dist` within a package called `probability`:

```python
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext

import numpy as np

setup(
    cmdclass = {'build_ext': build_ext},
    ext_modules = [Extension("dist", ["probability/src/dist.pyx"], include_dirs=[np.get_include()])]
)
```
    
It mainly uses machinery from a core Python package `distutils` that manages the build process.

To get a closer look at where Cython is improving our unchanged Python code, we can add an `--annotate` flag to the `%%cython` magic declaration:

In [ ]:
%%cython --annotate

def f(x):
    return 2*x*x + 3*x + 1
      
def trapez2(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h

In the above, the line color indicates the "typedness" of the extension, where yellower lines are closer to Python, and therefore require calls to the Python C API, while whiter lines indicate code that is closer to pure C, hence requiring few, if any, Python API calls.

If you click on a line, it unravels to show you the C code that results from the call to `cython`.

The goal in speeding up code with Cython is to turn as many lines to white as we can. The easiest way to do this is to add type declarations to the Python code:

In [ ]:
%%cython --annotate

# Add type to argument
def ff(double x):
    return 2*x*x + 3*x + 1

# Add types to arguments
def trapez3(double a, double b, int n):
    # Declare types of variables
    cdef double h, x, sumy
    cdef int i
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += ff(x)
    sumy += 0.5*(ff(a) + ff(b))
    return sumy*h

In [ ]:
%timeit df.apply(lambda x: trapez3(x.a, x.b, x.N), axis=1)

This gives us a considerable speedup. Let's have a look at the profiler report for the new function:

In [ ]:
%prun -l 4 df.apply(lambda x: trapez3(x.a, x.b, x.N), axis=1)

The next thing we might try is to *inline* the polynomial function. By inlining, we mean that we ask the compiler to perform an inline expansion of said function; that is, it will insert a copy of the function itself wherever the function is called, instead of calling the function wherever it is defined.

We do three things to the specification of `ff`:

* change `def` to `cdef`
* add a return type to the function
* add an `inline` keyword

In [ ]:
%%cython --annotate

cdef inline double ff(double x):
    return 2*x*x + 3*x + 1

cpdef trapez4(double a, double b, int n):
    cdef double h, x, sumy
    cdef int i
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += ff(x)
    sumy += 0.5*(ff(a) + ff(b))
    return sumy*h

The `cdef` keyword declares a C object. Everything that follows it is therefore specified in terms of C; we are essentially writing C, but using a subset of Python's syntax rules. So, when we are creating a function `cdef ff` it is a C function, and is not available to you in Python.

`cpdef` is a hybrid declaration that creates both a C interface and a Python interface to the function.

Let's see how this performs.

In [ ]:
%timeit df.apply(lambda x: trapez4(x.a, x.b, x.N), axis=1)

Woof! That's a big speedup, and there's not much yellow left in the annotated code. 

If you would like a very simple way of injecting types into your code with Cython, without modifying any of the code itelf, you can use the `@cython.locals` decorator. Note that you don't get as fast of a speedup as we have just achieved.

In [ ]:
%%cython
import cython

@cython.locals(x=cython.double)
def f(x):
    return 2*x*x + 3*x + 1
     
@cython.locals(a=cython.double, b=cython.double, n=cython.int,
               h=cython.double, sumy=cython.double, i=cython.int,
               x=cython.double, func=cython.double)
def trapez5(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h

In [ ]:
%timeit df.apply(lambda x: trapez5(x.a, x.b, x.N), axis=1)

If you can stand to look at it, you can peek at all the C code that is generated by Cython just to optimize this short function.

In [ ]:
%load ../examples/trapezoid.c

### Using `ndarray`

If we profile the function now, our functions are not near the top of the list.

In [ ]:
%prun -l 4 df.apply(lambda x: trapez4(x.a, x.b, x.N), axis=1)

We notice, however, that `series` is being called a lot. 

Each row is being turned into a `Series`

In [ ]:
%%cython
cimport numpy as np
import numpy as np

cdef inline double ff(double x) except? -2:
    return 2*x*x + 3*x + 1

cpdef trapez4(double a, double b, int n):
    cdef double h, x, sumy
    cdef int i
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += ff(x)
    sumy += 0.5*(ff(a) + ff(b))
    return sumy*h

cpdef np.ndarray[double] apply_trapez(np.ndarray col_a, np.ndarray col_b, np.ndarray col_n):
    assert (col_a.dtype == np.float and col_b.dtype == np.float and col_n.dtype == np.int)
    
    cdef Py_ssize_t i, n = len(col_n)
    assert (len(col_a) == len(col_b) == n)
    cdef np.ndarray[double] res = np.empty(n)
    
    for i in range(len(col_a)):
        res[i] = trapez4(col_a[i], col_b[i], col_n[i])
    return res

In [ ]:
%timeit apply_trapez(df['a'].values, df['b'].values, df['N'].values)

This has cut the execution time yet again. We can see now that there appears to be little remaining to optimize.

In [ ]:
%prun -l 4 apply_trapez(df['a'].values, df['b'].values, df['N'].values)

### Compiler directives

Here's another simple example, using a function that calculates the Euclidean distance between two arrays:

In [ ]:
def euclidean(x, y):
    x = np.array(x)
    y = np.array(y)
    return np.sqrt(((x - y) ** 2).sum())

In [ ]:
%timeit euclidean(np.random.randn(10), np.random.randn(10))

In order to get a speedup under Cython, we need to iterate over the elements of each passed array, and aggregate them manually.

In [ ]:
%%cython --annotate

import cython
cimport numpy as np
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def euclidean2(np.ndarray[np.float64_t, ndim=1] x, 
               np.ndarray[np.float64_t, ndim=1] y):
    cdef: 
        double diff
        int i
    diff = 0
    for i in range(x.shape[0]):
        diff += (x[i] - y[i])**2
    return sqrt(diff)


In [ ]:
%timeit euclidean2(np.random.randn(10), np.random.randn(10))

The decorators for `euclidean2` are **compiler directives** that alter the behavior of Cython code. Setting `boundscheck` to False removes boundary checking for indexing operations, forcing us to ensure that we do not try to index arrays using index vlaues that are out of bounds. When we set `wraparound` to False, Cython will not support negative indexes, as is the case with Python. While these directives may increase the speed of our code, it can be dangerous; if we do not ensure that we index our arrays properly, it may cause segmentation faults or data corruption.

The full set of compiler directives are described in the [Cython docs](http://docs.cython.org/src/reference/compilation.html#compiler-directives).

Here is the same code using lists instead of NumPy arrays:

In [ ]:
%%cython --annotate

from libc.math cimport sqrt

def euclidean3(list x, list y):
    cdef: 
        double diff
        int i
    diff = 0
    for i in range(len(x)):
        diff += (x[i] - y[i])**2
    return sqrt(diff)


In [ ]:
%timeit euclidean3(np.random.randn(10).tolist(), np.random.randn(10).tolist())

### Exercise

Try using the compiler directives on the `apply_trapez` function to see if there are further performance gains to be had.

In [ ]:
# Write your answer here

## Exercise

Try using Cython to improve the performance of a gradient descent algorithm:

In [ ]:
from scipy import optimize

def gradient_descent(x0, f, f_prime, adapt=False):
    x_i, y_i = x0
    all_x_i = list()
    all_y_i = list()
    all_f_i = list()

    for i in range(1, 100):
        all_x_i.append(x_i)
        all_y_i.append(y_i)
        all_f_i.append(f([x_i, y_i]))
        dx_i, dy_i = f_prime(np.asarray([x_i, y_i]))
        if adapt:
            # Compute a step size using a line_search
            step = optimize.line_search(f, f_prime,
                                np.r_[x_i, y_i], -np.r_[dx_i, dy_i],
                                np.r_[dx_i, dy_i], c2=.05)
            step = step[0]
        else:
            step = 1
        x_i += -step*dx_i
        y_i += -step*dy_i
        if np.abs(all_f_i[-1]) < 1e-16:
            break
    return all_x_i, all_y_i, all_f_i

Here is a sample function to optimize. Recall from Section 3 that it returns both the quadratic function and its gradient.

In [ ]:
def quad(epsilon, ndim=2):
    def f(x):
        x = np.asarray(x)
        y = x.copy()
        y *= np.power(epsilon, np.arange(ndim))
        return .33*np.sum(y**2)

    def f_prime(x):
        x = np.asarray(x)
        y = x.copy()
        scaling = np.power(epsilon, np.arange(ndim))
        y *= scaling
        return .33*2*scaling*y

    return f, f_prime

In [ ]:
x0, y0 = 1.6, 1.1

f, f_prime = quad(0.8)
%timeit gd_x_i, gd_y_i, gd_f_i = gradient_descent([x0, y0], f, f_prime)

In [ ]:
# Write answer here

## Numba

Cython precompiles parts of Python code before running. Another approach is **Just-in-Time (JIT)** compilation. Numba is a compiler that runs Python code through an LLVM compiler to produce optimized bytecode for fast execution. Numba does not require a C/C++ compiler on your machine.

Numba's lone API is a **decorator**.

The `@jit` decorator runs the decorated function through bytecode analysis and the function arguments through a type inference engine, and generates an intermediate representation of your code, which is then passed to LLVM for compilation to bytecode.

In [ ]:
from numba import jit, autojit

In [ ]:
@jit
def nfibonacci(size):
    F = np.empty(size, 'int')
    a, b = 0, 1
    for i in range(size):
        F[i] = a
        a, b = b, a + b
    return F

In [ ]:
nfibonacci(50)

Numba is able to compile separate specializations depending on the input types.

If you want fine-grained control over types chosen by the compiler, you can tell Numba the function signature (types) to expect.

In [ ]:
from numba import int32

@jit(int32[:](int32))
def nfibonacci(size):
    F = np.empty(size, 'int')
    a, b = 0, 1
    for i in range(size):
        F[i] = a
        a, b = b, a + b
    return F

In [ ]:
nfibonacci(50)

Compilation is deferred until the first function execution. Numba will infer the argument types at call time, and generate optimized code based on this information. 

In [ ]:
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

X = np.random.random((1000, 3))

%timeit pairwise_python(X)

In [ ]:
@jit
def npairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

In [ ]:
%timeit npairwise_python(X)

Numba-compiled functions can call other compiled functions. In some situations, the optimizer may even inline the function in the machine code code.

In [ ]:
def square(x):
    return x ** 2

def hypot(x, y):
    return np.sqrt(square(x) + square(y))

In [ ]:
%timeit hypot(10, 8)

In [ ]:
@jit
def nsquare(x):
    return x ** 2

@jit
def nhypot(x, y):
    return np.sqrt(nsquare(x) + nsquare(y))

In [ ]:
%timeit nhypot(10, 8)

Numba can compile *most* NumPy functions, as well as generators.

Numba does *not* compile things like lists, sets, dictionaries (tuples are compiled), comprehensions, and string operations, so there will be no speedup for these.

As with all performance tools, the best strategy is not to apply the `@jit` decorator all over your code, but to use Python's profiling tools to identify "hotspots" in your program, and selectively apply `@jit`.

Numba can also be used to **vectorize** computations, meaning that one does not explictly have to loop over iterables of values.

In [ ]:
from numba import vectorize

@vectorize
def nsquare_vec(x):
    return x**2

In [ ]:
df

In [ ]:
%timeit nsquare(df.a)

In [ ]:
%timeit nsquare_vec(df.a.values)

Note that as of pandas 0.20, Numba only works on the underlying array in pandas data structures, and not DataFrames or Series themselves, hence the passing of `.values` above.

One performance caveat is that Numba will only speed up code that uses NumPy arrays (so-called `nopython` mode). When your code includes things like lists, strings or dictionaries, it will revert to `object` mode and not provide an appreciable speedup to your code. If you wish to have an exception thrown when `object` mode is used, you can apply the following argument to the `jit` decorator:

    @jit(nopython=True)

### Exercise

Use Numba to just-in-time compile the trapezoidal integration function we used above. See how it compares with Cython.

In [ ]:
# Write your answer here

## References

Augspurger, T. (2016) [Effective Pandas](https://leanpub.com/effective-pandas). Leanpub.

van der Plas, J. (2014) [Why Python is Slow: Looking Under the Hood](https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/)

van der Plas, J. (2015) [Optimizing Python in the Real World: NumPy, Numba, and the NUFFT](https://jakevdp.github.io/blog/2015/02/24/optimizing-python-with-numpy-and-numba/)

[A guide to analyzing Python performance](http://www.huyng.com/posts/python-performance-analysis/)

[Kurt Smith's Cython tutorial from SciPy 2013](https://www.youtube.com/watch?v=JKCjsRDffXo)
